In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing

import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('future-gc00-daily-prices.csv', parse_dates=['Date'], index_col='Date')
# Ensure the 'Close' column is in float format
df['Close'] = df['Close'].str.replace(',', '').astype(float)

# Split the data into training and test sets
train = df['Close'][:int(0.8*len(df))]
test = df['Close'][int(0.8*len(df)):]

# Apply ARIMA model
arima_model = ARIMA(train, order=(5, 1, 0))
arima_fit = arima_model.fit()
arima_forecast = arima_fit.forecast(steps=len(test))

# Apply Exponential Smoothing models
# Simple Exponential Smoothing
ses_model = ExponentialSmoothing(train, trend=None, seasonal=None)
ses_fit = ses_model.fit()
ses_forecast = ses_fit.forecast(steps=len(test))

# Double Exponential Smoothing (Holt's Linear Trend Model)
des_model = ExponentialSmoothing(train, trend='add', seasonal=None)
des_fit = des_model.fit()
des_forecast = des_fit.forecast(steps=len(test))

# Triple Exponential Smoothing (Holt-Winters Seasonal Model)
tes_model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
tes_fit = tes_model.fit()
tes_forecast = tes_fit.forecast(steps=len(test))

# Plot the results
plt.figure(figsize=(14, 8))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, arima_forecast, label='ARIMA Forecast')
plt.plot(test.index, ses_forecast, label='SES Forecast')
plt.plot(test.index, des_forecast, label='DES Forecast')
plt.plot(test.index, tes_forecast, label='TES Forecast')
plt.legend(loc='best')
plt.title('Gold Future Prices Forecast')
plt.show()

# Compute residuals
arima_residuals = test - arima_forecast
ses_residuals = test - ses_forecast
des_residuals = test - des_forecast
tes_residuals = test - tes_forecast

# Print residuals
print("ARIMA Residuals:\n", arima_residuals)
print("SES Residuals:\n", ses_residuals)
print("DES Residuals:\n", des_residuals)
print("TES Residuals:\n", tes_residuals)

# Explanation of the algorithms
explanation = """
Box-Jenkins ARIMA:
ARIMA stands for AutoRegressive Integrated Moving Average. It combines three components:
- AR (AutoRegressive): Uses the dependency between an observation and a number of lagged observations.
- I (Integrated): Uses differencing of raw observations to make the time series stationary.
- MA (Moving Average): Uses dependency between an observation and a residual error from a moving average model applied to lagged observations.

Exponential Smoothing:
Exponential Smoothing methods apply weighted averages of past observations, with the weights decaying exponentially as the observations get older.
- Simple Exponential Smoothing (SES): Suitable for data without trend or seasonality.
- Double Exponential Smoothing (DES): Suitable for data with a trend but no seasonality.
- Triple Exponential Smoothing (TES): Suitable for data with both trend and seasonality.
"""
print(explanation)